<a href="https://colab.research.google.com/github/simecek/PseudoDNA_Generator/blob/master/data/Random_3UTR_Seqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Random 3'UTR Sequences


## Setup

Installation for colab environment.

In [ ]:
!pip install biopython pyensembl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.2/537.2 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 4.1 MB/s eta 0:00:00
  Created wheel for memoized-property: filename=memoized_property-1.0.3-py2.py3-none-any.whl size=4184 sha256=dc80a7ded26633cb3434e9399c506fdd6b709241e266a3dee8bebbab5187af8c
  Stored in 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pyensembl install --release 97 --species human

2024-05-27 23:15:24,740 - pyensembl.shell - INFO - Running 'install' for EnsemblRelease(release=97, species='homo_sapiens')
2024-05-27 23:15:24,740 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz from URL https://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz
2024-05-27 23:15:24,741 - datacache.download - INFO - Downloading https://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz to /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz
2024-05-27 23:15:27,020 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL https://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
2024-05-27 23:15:27,021 - datacache.download - INFO - Downloading https://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz to

In [ ]:
import pandas as pd
import numpy as np
import gzip
from tqdm.notebook import tqdm

from Bio import SeqIO   # for reading fasta files
from pyensembl import EnsemblRelease   # to get the gene list
import random

ENSEMBL_RELEASE = 97
DNA_TOPLEVEL_FASTA_PATH = "/content/drive/My Drive/data/ensembl/Homo_sapiens.GRCh38.dna.toplevel.fa.gz"

# to generate random sequences
N = 15000    # how many
K = [random.randint(200, 500) for _ in range(15000)]
OUTPUT_FILE = '/content/drive/My Drive/data/random/3utr.csv'

CHRS = [str(chr) for chr in range(1,23)] + ['X', 'Y', 'MT']

## Get transcript list

In [ ]:
# release 97 uses human reference genome GRCh38
data = EnsemblRelease(ENSEMBL_RELEASE)

In [ ]:
human_transcripts = data.transcript_ids()
len(human_transcripts)

226788

In [ ]:
human_transcripts[0], data.transcript_by_id(human_transcripts[0])


('ENST00000000233',
 Transcript(transcript_id='ENST00000000233', transcript_name='ARF5-201', gene_id='ENSG00000004059', biotype='protein_coding', contig='7', start=127588411, end=127591700, strand='+', genome='GRCh38'))

In [ ]:
transcripts_full_info  = [data.transcript_by_id(transcript) for transcript in human_transcripts]

In [ ]:
human_transcript_tuples = [(x.transcript_id, x.gene_id, x.biotype, x.contig, x.start, x.end, x.strand, x.three_prime_utr_sequence) for x in transcripts_full_info if x.contains_start_codon & x.contains_stop_codon]
human_transcript_table = pd.DataFrame.from_records(human_transcript_tuples, columns=["id", "gene_id", "biotype", "chr", "start", "end", "strand", "three_prime_utr_sequence"])
assert all(human_transcript_table.start <= human_transcript_table.end)

human_transcript_table.head()

,id,gene_id,biotype,chr,start,end,strand,three_prime_utr_sequence
0,ENST00000000233,ENSG00000004059,protein_coding,7,127588411,127591700,+,CCAGCCAGGGGCAGGCCCCTGATGCCCGGAAGCTCCTGCGTGCATC...
1,ENST00000000412,ENSG00000003056,protein_coding,12,8940361,8949645,-,ATTGCACTTTATATGTCCAGCCTCTTCCTCAGTCCCCCAAACCAAA...
2,ENST00000000442,ENSG00000173153,protein_coding,11,64305524,64316743,+,GGCAAGGGGTGGGACTGGTGGGGGTTCTGGCAGGACCTGCCTAGCA...
3,ENST00000001008,ENSG00000004478,protein_coding,12,2794970,2805423,+,CCCCTCTCCACCAGCCCTACTCCTGCGGCTGCCTGCCCCCCAGTCT...
4,ENST00000001146,ENSG00000003137,protein_coding,2,72129238,72148038,-,CCCAAGACCCACCCGCCTCAGCCCAGCCCAGGCAGCGGGGTGGTGG...


In [ ]:
print(human_transcript_table['three_prime_utr_sequence'].iloc[3])

CCCCTCTCCACCAGCCCTACTCCTGCGGCTGCCTGCCCCCCAGTCTCCCCACTCCACCCTGTTAGTTTTGTAAAAACTGAAGAATTTTGAGTGAATTAGACCTTTATTTTTCTATCTGGTTGGATGGTGGCTTTAGGGGAAGGGGGAAAGGTGTAGGCTGGGGGATTGAGGTGGGGAATCATTTTAGCTGGTGTCAGCCCCTCTTCCCTTCCTCCATTGCACATGAACATATGTCCATCCATATATATTCATCAGAATGTTAATTTATTTTGCTCCCTCTGTTAGGTCCATTTTCTAAGGGTAGAAGAGGCAAGTGGTAGGGATGAGGTCTGATAAGAACCCAGGGTGGAGAGGGAGACTCCTGGGCAGCCGTTTTCCTCATCCTTTCCCTCTCCCAGTCCATTTCCAAATGTGGCCTCCATGTGGGTGCTAGGGACATGGGAAAAACCACTGCTATGCCATTTCTTCTCTCTGTTCCCTTCCTCACCCCCGACGGTGTGGCTGATGATGTCTTCTGGTGTCATGGTGACCACCCCCTGTTCCCTGTTCTGGTATTTCCCCTGTCAGTTTCCCCTCTCGGCCAGGTTGTGTCCCAAAATCCCCTCAGCCTCTTCTCTGCACGTTGCTGAAGGTCCAGGCTTGCCTCAAGTTCCATGCTTGAGCAATAAAGTGGAAACAATAAAACCTGGGTGTCAGACAACCCTTTCTGTTCAGCCTTGGAGTGGTGGGTATGGGTGGGTACATAATGGGTAGTAGCACAATCAAGGGGTCACCCACTTAGTTCCAGTTGAGCTTAAATAAGTGAGCATCTCATGTAGATGCCATAGTTGGGCAGGAAGCAGCCCCACCATGCATGGCTTCCTGAGCACTGCACAGGCTGCCGCTGGGATTTGTGTCTATGGCTTCGCCCAAGGTTCCCAGCACTCCCTCCCTCCAGCAACCTGGCAGTTTTAGTGCCTCTGGTTTTCTTCCCTGACACTTAGGAAGACGAAAGTATAAA

In [ ]:
assert ~human_transcript_table.three_prime_utr_sequence.str.contains('N').any()

human_transcript_table['length'] = human_transcript_table.three_prime_utr_sequence.apply(len)
selected_regions = human_transcript_table[human_transcript_table.length > 500].copy()
selected_regions = selected_regions[selected_regions.strand == '+'].copy()
human_transcript_table.shape, selected_regions.shape

((68734, 9), (18637, 9))

In [ ]:
# sample_regions = selected_regions.sample(N)
# sample_regions['random_start'] = [np.random.randint(c_len - K[i]) for i, c_len in enumerate(sample_regions.length)]
# sample_regions['random_end'] = sample_regions['random_start'] + K - 1
# sample_regions['seq'] = ''
# for i in range(sample_regions.shape[0]):
#   sample_regions['seq'].iloc[i] = (sample_regions['three_prime_utr_sequence'].iloc[i])[sample_regions['random_start'].iloc[i]:sample_regions['random_start'].iloc[i]+K[i]]
# sample_regions.head()
sample_regions = selected_regions.sample(N)
sample_regions['random_end'] = sample_regions['end']
sample_regions['random_start'] = sample_regions['random_end'] - K + 1
sample_regions['seq'] = ''
for i in range(sample_regions.shape[0]):
  sample_regions['seq'].iloc[i] = (sample_regions['three_prime_utr_sequence'].iloc[i])[-K[i]:]
sample_regions.head()

Streaming output truncated to the last 5000 lines.
<ipython-input-40-876ec2565ab8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_regions['seq'].iloc[i] = (sample_regions['three_prime_utr_sequence'].iloc[i])[-K[i]:]
<ipython-input-40-876ec2565ab8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_regions['seq'].iloc[i] = (sample_regions['three_prime_utr_sequence'].iloc[i])[-K[i]:]
<ipython-input-40-876ec2565ab8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

,id,gene_id,biotype,chr,start,end,strand,three_prime_utr_sequence,length,random_end,random_start,seq
16353,ENST00000359866,ENSG00000130311,protein_coding,19,17309563,17323298,+,GACTCTCAACTCCACAGGCGCCTCCTGCCAGGTCTGCTCCTCGGTC...,3642,17323298,17323036,ATCTGGGCCGTCTCAGAGGCGGCGCAGCCTGGAGTTTTCCATCTGT...
5563,ENST00000284878,ENSG00000154639,protein_coding,21,17513043,17570100,+,AGCCTCCATATGTCTCATCTGTGCTCTCCGTGTTCCTTTCCTTTTT...,4408,17570100,17569747,GTTTTGGTCATCGAGACTAAAAGCTCCATCAAAACAGAACTTTGTG...
34097,ENST00000433147,ENSG00000100150,protein_coding,22,31753955,31906998,+,GGCCAGGCTGCACCTGTGCTGGGGGAAGGTGGGTGAGCCACTGCCC...,501,31906998,31906509,ACCTGTGCTGGGGGAAGGTGGGTGAGCCACTGCCCTCAAACCCGGG...
11644,ENST00000334056,ENSG00000163806,protein_coding,2,28810834,28850610,+,GATGGCCCAACTAAACCAATTTGGAATCATTAACTACAAAATGTCA...,669,28850610,28850405,TCTGTTGTAAAAAAATATATGTACTATAAGCTACATAAAATATTTT...
43539,ENST00000510658,ENSG00000064692,nonsense_mediated_decay,5,122311691,122463954,+,GGATTTTCCAAGCCTTATTCATTACGCAGGTTGCTATGGCCAGTCA...,2020,122463954,122463664,CAATGCCTCACCAGCAGAAGAACAGAATATCAGGATGCCTTAAATT...


## Random transcript selection

In [ ]:
sample_regions.shape

(15000, 12)

In [ ]:
seqs = sample_regions[['id', 'chr', 'start', 'end', 'random_start', 'random_end', 'seq']].copy().reset_index(drop=True)
seqs.head()

,id,chr,start,end,random_start,random_end,seq
0,ENST00000359866,19,17309563,17323298,17323036,17323298,ATCTGGGCCGTCTCAGAGGCGGCGCAGCCTGGAGTTTTCCATCTGT...
1,ENST00000284878,21,17513043,17570100,17569747,17570100,GTTTTGGTCATCGAGACTAAAAGCTCCATCAAAACAGAACTTTGTG...
2,ENST00000433147,22,31753955,31906998,31906509,31906998,ACCTGTGCTGGGGGAAGGTGGGTGAGCCACTGCCCTCAAACCCGGG...
3,ENST00000334056,2,28810834,28850610,28850405,28850610,TCTGTTGTAAAAAAATATATGTACTATAAGCTACATAAAATATTTT...
4,ENST00000510658,5,122311691,122463954,122463664,122463954,CAATGCCTCACCAGCAGAAGAACAGAATATCAGGATGCCTTAAATT...


In [ ]:
len(seqs.seq.values[1]), seqs.seq.values[1]

(354,
 'GTTTTGGTCATCGAGACTAAAAGCTCCATCAAAACAGAACTTTGTGTTTTCTGCTAACTTATTTAATGACACAAGTTTTAAGAGAACCACAATTCATTGATTCACTTATTCTTTTCCCTAATTGTGAATTTTAGTGATAAATACACCTGTACTACTGAGGAAAATATTCTGACACTTCACGTGTGCAAAGTATAGAACTGACAGTGTCAGTTTCAGATTTTGTATGTACGATTTCTGGCTTATATATCCAATGGTGCAGATTTTGAAATTTGTAAGAACAAAATTTGTTAAGAAAAACAACTTGCTCTAGTTTTGTGACCTTGTGTACTTTTGAAATAAAATCAAGAAAGCAGT')

## Save generated sequences to file

In [ ]:
seqs.to_csv('/content/drive/My Drive/genomic_data/3utr.csv', index=False)